In [1]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import numpy as np
import os
import pandas as pd
from pathlib import Path
from collections import Counter

In [11]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [47]:
columns = ["budget", "popularity", "release_date", "revenue","runtime", "title", "vote_average"]
    


target = ["revenue"]

In [49]:
# Load the data
file_path = Path('clean_movies.csv')
movies_df = pd.read_csv(file_path,)[:-2]
#df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
#movies_df = df.dropna(axis='columns', how='all')

# Drop the null rows
#movies_df = df.dropna()

# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_earner'}   
movie_df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_earner')    
movie_df = df.replace(x)

movie_df.reset_index(inplace=True, drop=True)




movies_df.head()


,budget,popularity,release_date,revenue,runtime,title,vote_average
0,30000000,21.946943,1995-10-30,373554033.0,81.0,Toy Story,7.7
1,65000000,17.015539,1995-12-15,262797249.0,104.0,Jumanji,6.9
2,16000000,3.859495,1995-12-22,81452156.0,127.0,Waiting to Exhale,6.1
3,60000000,17.924927,1995-12-15,187436818.0,170.0,Heat,7.7
4,58000000,6.677277,1995-12-15,0.0,127.0,Sabrina,6.2


Split the Data into Training and Testing

In [50]:
# Create our features
X = movies_df.drop("revenue", axis=1)

X = pd.get_dummies(X)

# Create our target
y = movies_df["revenue"]

In [51]:
X.describe()

,budget,popularity,runtime,vote_average,release_date_1902-09-01,release_date_1903-12-01,release_date_1904-10-01,release_date_1905-07-03,release_date_1914-06-01,release_date_1915-01-01,...,title_iBoy,title_iNumber Number,title_xXx,title_xXx: Return of Xander Cage,title_xXx: State of the Union,title_¡Three Amigos!,title_Æon Flux,title_Іван Сила,title_Королёв,title_Юленька
count,8.887000e+03,8887.000000,8877.000000,8887.000000,8887.000000,8887.000000,8887.000000,8887.000000,8887.000000,8887.000000,...,8887.000000,8887.000000,8887.000000,8887.000000,8887.000000,8887.000000,8887.000000,8887.000000,8887.000000,8887.000000
mean,2.161070e+07,7.369227,105.387406,6.014156,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,...,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113,0.000113
std,3.431462e+07,11.431103,28.295153,1.246546,0.010608,0.010608,0.010608,0.010608,0.010608,0.010608,...,0.010608,0.010608,0.010608,0.010608,0.010608,0.010608,0.010608,0.010608,0.010608,0.010608
min,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000e+06,2.151209,91.000000,5.400000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,8.000000e+06,6.324751,101.000000,6.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.500000e+07,10.016471,116.000000,6.800000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.800000e+08,547.488298,705.000000,10.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [52]:
# Check the balance of our target values
y.value_counts()

0.0            3508
12000000.0       16
10000000.0       15
2000000.0        15
11000000.0       15
               ... 
37664855.0        1
12918858.0        1
181001478.0       1
43061982.0        1
1328612.0         1
Name: revenue, Length: 5007, dtype: int64

In [53]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

Ensemble Learners
In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

Train the model using the training data.
Calculate the balanced accuracy score from sklearn.metrics.
Print the confusion matrix from sklearn.metrics.
Generate a classication report using the imbalanced_classification_report from imbalanced-learn.
For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score
Note: Use a random state of 1 for each algorithm to ensure consistency between tests

Balanced Random Forest Classifier

In [54]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
random_forest = BalancedRandomForestClassifier(n_estimators = 100)
random_forest = random_forest.fit(X_train, y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score
y_pred = random_forest.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

In [ ]:
# List the features sorted in descending order by feature importance
feature_names = X.columns
sorted(zip(random_forest.feature_importances_, feature_names), reverse=True)

Easy Ensemble AdaBoost Classifier

In [ ]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier
easy = EasyEnsembleClassifier(n_estimators = 100,random_state=1)
easy = easy.fit(X_train, y_train)

In [ ]:
# Calculated the balanced accuracy score
y_pred = easy.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))